<a href="https://colab.research.google.com/github/Soochin/cs175-project/blob/master/notebooks/SkinDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/School/Year\ 4/Winter/CS\ 175/Final Project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/School/Year 4/Winter/CS 175/Final Project


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import random
import numpy as np
import io
import os
import timeit
from PIL import Image


In [0]:
#Create giant tensor files from hand images
#ONLY DO IF THE .pt FILES HAVE NOT YET BEEN CREATED
transform = T.ToTensor()
i=0
ts = []
outputBatch =0 
for handFile in os.listdir('./data/skin_detection_data/skin_detected_hands_128'):
  t = transform(Image.open('./data/skin_detection_data/skin_detected_hands_128/'+handFile))
  ts.append(t)
  i += 1
  if i%50 == 0:
    print(i, end=',')
  if i%1000 == 0:
    print()
    t_stacked = torch.stack(ts)
    ts = []
    print(t_stacked.size())
    torch.save(t_stacked, './data/skin_detection_data/hands/batch_'+str(outputBatch)+'.pt')
    outputBatch += 1
    t_stacked = None
t_stacked = torch.stack(ts)
ts = []
print(t_stacked.size())
torch.save(t_stacked, './data/skin_detection_data/hands/batch_'+str(outputBatch)+'.pt')
t_stacked = None

#Clean out memory
ts = []
t_stacked = None
i = 0
t = None

50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,
torch.Size([1000, 3, 128, 128])
1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,
torch.Size([1000, 3, 128, 128])
2050,2100,2150,2200,2250,2300,2350,2400,2450,2500,2550,2600,2650,2700,2750,2800,2850,2900,2950,3000,
torch.Size([1000, 3, 128, 128])
3050,3100,3150,3200,3250,3300,3350,3400,3450,torch.Size([498, 3, 128, 128])


In [27]:
#Create giant tensor files from nohand images
#ONLY DO IF THE .pt FILES HAVE NOT YET BEEN CREATED
transform = T.ToTensor()
i=0
ts = []
outputBatch =0 
for handFile in os.listdir('./data/skin_detection_data/skin_detected_nohands_128'):
  t = transform(Image.open('./data/skin_detection_data/skin_detected_nohands_128/'+handFile))
  ts.append(t)
  i += 1
  if i%50 == 0:
    print(i, end=',')
  if i%1000 == 0:
    print()
    t_stacked = torch.stack(ts)
    ts = []
    print(t_stacked.size())
    torch.save(t_stacked, './data/skin_detection_data/nohands/batch_'+str(outputBatch)+'.pt')
    outputBatch += 1
    t_stacked = None
t_stacked = torch.stack(ts)
ts = []
print(t_stacked.size())
torch.save(t_stacked, './data/skin_detection_data/nohands/batch_'+str(outputBatch)+'.pt')
t_stacked = None

#Clean out memory
ts = []
t_stacked = None
i = 0
t = None

50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,
torch.Size([1000, 3, 128, 128])
1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,
torch.Size([1000, 3, 128, 128])
2050,2100,2150,2200,2250,2300,2350,2400,2450,2500,2550,2600,2650,2700,2750,2800,2850,2900,2950,3000,
torch.Size([1000, 3, 128, 128])
3050,3100,3150,3200,3250,3300,3350,3400,3450,3500,3550,3600,3650,3700,3750,3800,3850,3900,3950,4000,
torch.Size([1000, 3, 128, 128])
4050,4100,4150,4200,torch.Size([220, 3, 128, 128])


In [0]:
class HandClassifyDataset(torch.utils.data.Dataset):
  def __init__(self, rootdir):
    self.rootdir = rootdir
    self.handsdir = self.rootdir+'hands/'
    self.nohandsdir = self.rootdir+'nohands/'
    self.handIms = []
    self.nohandIms = []
    i = 0
    for batch in os.listdir(self.handsdir):
      if "batch" in batch:
        t = torch.load(self.handsdir+batch)
        print(t.size())
        self.handIms.append(t)
        print('Batch done')
    self.handIms = torch.cat(self.handIms, 0)
    print(self.handIms.size())
    for batch in os.listdir(self.nohandsdir):
      if "batch" in batch:
        t = torch.load(self.nohandsdir+batch)
        print(t.size())
        self.nohandIms.append(t)
        print('Batch done')
    self.nohandIms = torch.cat(self.nohandIms, 0)
    print(self.nohandIms.size())
    self.ind = [x for x in range(self.handIms.size()[0]+ self.nohandIms.size()[0])]
    random.shuffle(self.ind)
    
  def __len__(self):
    return self.handIms.size()[0]+self.nohandIms.size()[0]
  
  def __getitem__(self, idx):
    idx = self.ind[idx]
    if idx < self.handIms.size()[0]:
      sample = (self.handIms[idx,:,:,:], 1)
      return sample
    elif idx < self.handIms.size()[0] + self.nohandIms.size()[0]:
      idx = idx-self.handIms.size()[0]
      sample = (self.nohandIms[idx,:,:,:], 0)
      #print(handBoxes[handFiles[idx]].size())
      return sample
    else:
      raise IndexError

In [30]:
hands_dataset = None
hands_dataset = HandClassifyDataset(rootdir='./data/skin_detection_data/')
print(len(hands_dataset))

torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([498, 3, 128, 128])
Batch done
torch.Size([3498, 3, 128, 128])
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([1000, 3, 128, 128])
Batch done
torch.Size([220, 3, 128, 128])
Batch done
torch.Size([4220, 3, 128, 128])
7718


In [0]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

hands_dataset_size = len(hands_dataset)
NUM_VAL = int(np.ceil(0.12*hands_dataset_size))
NUM_TEST = int(np.ceil(0.13*hands_dataset_size))
NUM_TRAIN = len(hands_dataset)-NUM_VAL-NUM_TEST

In [0]:
train_loader = torch.utils.data.DataLoader(hands_dataset, batch_size=64, num_workers=4, sampler=ChunkSampler(NUM_TRAIN, 0))
val_loader = torch.utils.data.DataLoader(hands_dataset, batch_size=64, num_workers=4, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))
test_loader = torch.utils.data.DataLoader(hands_dataset, batch_size=64, num_workers=4, sampler=ChunkSampler(NUM_TEST, NUM_TRAIN+NUM_VAL))

In [0]:
import copy
gpu_dtype = torch.cuda.FloatTensor
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def train(model, loss_fn, optimizer, num_epochs = 1):
  train_acc = list()
  val_acc = list()
  models = list()

  loss_tr = 0

  total_step = len(train_loader)
  for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      #Forward pass
      outputs = model(images)
      loss_tr = loss_fn(outputs, labels)

      #Backward and optimize
      optimizer.zero_grad()
      loss_tr.backward()
      optimizer.step()

      if (i+1)%30 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
             .format(epoch+1, num_epochs, i+1, total_step, loss_tr.item()))
    acc = check_accuracy(model, val_loader)
    train_acc.append(loss_tr.item())
    val_acc.append(acc)
    models.append(model)
  best = val_acc.index(max(val_acc))
  print('Best Model: Train Accuracy {}, Test Accuracy {}'.format(train_acc[best]*100, val_acc[best]*100))
  return models[best]

def check_accuracy(model, loader):   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Test Accuracy of the model: Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

# Constant to control how frequently we print train loss
print_every = 10

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [66]:
#Simple model to test if DataLoader works
simple_model = nn.Sequential(
                nn.Conv2d(3, 128, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                Flatten(), # see above for explanation
                nn.Linear(476288, 2), # affine layer
              )
# Set the type of all data in this model to be FloatTensor 
simple_model.type(gpu_dtype)

loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.Adam(simple_model.parameters(), lr=1e-2) # lr sets the learning rate of the optimizer

simple_model.apply(reset)
train(simple_model, loss_fn, optimizer, num_epochs=1)
check_accuracy(simple_model, val_loader)

Epoch [1/1], Step [30/91], Loss: 0.6636
Epoch [1/1], Step [60/91], Loss: 0.6951
Epoch [1/1], Step [90/91], Loss: 0.6913


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Accuracy of the model: Got 474 / 927 correct (51.13)
Best Model: Train Accuracy 69.94981169700623, Test Accuracy 51.1326860841424
Test Accuracy of the model: Got 474 / 927 correct (51.13)


0.511326860841424

In [69]:
#Same model as used in general convolutional base model (control)

learning_rate = 1e-3

class GenConvNet(nn.Module):
  def __init__(self, num_classes=2):
    super(GenConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(32768, num_classes)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

  
controlModel = GenConvNet().to(device)
loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = torch.optim.Adam(controlModel.parameters(), lr=learning_rate)

best_model = train(controlModel, loss_fn, optimizer, num_epochs=10)
check_accuracy(best_model, test_loader)

#Using the optimized parameters for the regular images we struggle to break 65% accuracy

Epoch [1/10], Step [30/91], Loss: 0.7758
Epoch [1/10], Step [60/91], Loss: 0.8459
Epoch [1/10], Step [90/91], Loss: 0.7281


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Accuracy of the model: Got 474 / 927 correct (51.13)
Epoch [2/10], Step [30/91], Loss: 0.7240
Epoch [2/10], Step [60/91], Loss: 0.8426
Epoch [2/10], Step [90/91], Loss: 0.6348
Test Accuracy of the model: Got 536 / 927 correct (57.82)
Epoch [3/10], Step [30/91], Loss: 0.6554
Epoch [3/10], Step [60/91], Loss: 0.8175
Epoch [3/10], Step [90/91], Loss: 0.5866
Test Accuracy of the model: Got 469 / 927 correct (50.59)
Epoch [4/10], Step [30/91], Loss: 0.5884
Epoch [4/10], Step [60/91], Loss: 0.6151
Epoch [4/10], Step [90/91], Loss: 0.5576
Test Accuracy of the model: Got 552 / 927 correct (59.55)
Epoch [5/10], Step [30/91], Loss: 0.5909
Epoch [5/10], Step [60/91], Loss: 0.7317
Epoch [5/10], Step [90/91], Loss: 0.4948
Test Accuracy of the model: Got 566 / 927 correct (61.06)
Epoch [6/10], Step [30/91], Loss: 0.5118
Epoch [6/10], Step [60/91], Loss: 0.6090
Epoch [6/10], Step [90/91], Loss: 0.4362
Test Accuracy of the model: Got 605 / 927 correct (65.26)
Epoch [7/10], Step [30/91], Loss: 0.6

0.549800796812749

In [76]:
#Specialized Model for Preprocessed Images

learning_rate = 1e-3

class SkinConvNet(nn.Module):
  def __init__(self, num_classes=2):
    super(SkinConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = nn.Sequential(
      nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(16384, num_classes)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

  
skinModel = SkinConvNet().to(device)
loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = torch.optim.Adam(skinModel.parameters(), lr=learning_rate)

best_model = train(skinModel, loss_fn, optimizer, num_epochs=5)
check_accuracy(best_model, test_loader)

#Using the optimized parameters for the regular images we struggle to break 65% accuracy

Epoch [1/5], Step [30/91], Loss: 0.7815
Epoch [1/5], Step [60/91], Loss: 0.8032
Epoch [1/5], Step [90/91], Loss: 0.7019


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Accuracy of the model: Got 509 / 927 correct (54.91)
Epoch [2/5], Step [30/91], Loss: 0.6892
Epoch [2/5], Step [60/91], Loss: 0.7944
Epoch [2/5], Step [90/91], Loss: 0.6146
Test Accuracy of the model: Got 590 / 927 correct (63.65)
Epoch [3/5], Step [30/91], Loss: 0.7634
Epoch [3/5], Step [60/91], Loss: 0.7204
Epoch [3/5], Step [90/91], Loss: 0.5630
Test Accuracy of the model: Got 609 / 927 correct (65.70)
Epoch [4/5], Step [30/91], Loss: 0.7603
Epoch [4/5], Step [60/91], Loss: 0.6722
Epoch [4/5], Step [90/91], Loss: 0.5134
Test Accuracy of the model: Got 521 / 927 correct (56.20)
Epoch [5/5], Step [30/91], Loss: 0.6177
Epoch [5/5], Step [60/91], Loss: 0.6376
Epoch [5/5], Step [90/91], Loss: 0.4939
Test Accuracy of the model: Got 603 / 927 correct (65.05)
Best Model: Train Accuracy 68.70242953300476, Test Accuracy 65.69579288025889
Test Accuracy of the model: Got 687 / 1004 correct (68.43)


0.6842629482071713